# TP 3 et 4 plan d'exécution de requêtes

date de modification: 01/02/2024

BINOME

NOM 1: ZHOU

Prénom 1: runlin

Numéro : 28717281

NOM 2: ZHANG 

Prénom 2: zhile

Numéro : 21201131

# Préparation

installer DuckDB


In [1]:
# duckdb est déjà installé sur les machines de colab
# !pip install duckdb --pre

Les lib nécessaires pour ce TP

In [2]:
import duckdb

import sqlite3

import time
import pandas as pd
dataStroePath = "temp/"

## Créer les données


On utilise la base de données du benchmark TPC-H.
DuckDB propose la procedure *dbgen* pour générer la base de données.
Les données sont stockées dans le fichier *tpch.db*


Comprendre le [schéma des données TPC-H](https://nuage.lip6.fr/s/RramHt5W3RomySs) (ou cf moodle).

La taille de la base est paramétrée par le scale factor *sf=0.2*


In [4]:
# db = duckdb.connect(':memory:')
db = duckdb.connect('tpch.db')
db.execute("CALL dbgen(sf=0.1)")

## Consulter la BD

Il y a 8 tables. La commande `SHOW TABLES` retourne le nom des tables.

Note: la fonction  `.df()` ajoutée à la fin de l'expression sert à importer le résultat de la requête dans un dataframe pandas afin d'avoir un rendu "ergonomique" dans colab .

In [5]:
db.execute("SHOW TABLES").df()

,name
0,customer
1,lineitem
2,nation
3,orders
4,part
5,partsupp
6,region
7,supplier


La commande `DESCRIBE` retourne une description des attributs d'une table.

Exemple affichant les attributs de la table *Lineitem* :

In [6]:
db.execute("DESCRIBE lineitem").df()

,column_name,column_type,null,key,default,extra
0,l_orderkey,INTEGER,NO,None,None,NaN
1,l_partkey,INTEGER,NO,None,None,NaN
2,l_suppkey,INTEGER,NO,None,None,NaN
3,l_linenumber,INTEGER,NO,None,None,NaN
4,l_quantity,"DECIMAL(15,2)",NO,None,None,NaN
5,l_extendedprice,"DECIMAL(15,2)",NO,None,None,NaN
6,l_discount,"DECIMAL(15,2)",NO,None,None,NaN
7,l_tax,"DECIMAL(15,2)",NO,None,None,NaN
8,l_returnflag,VARCHAR,NO,None,None,NaN
9,l_linestatus,VARCHAR,NO,None,None,NaN


La clause  `LIMIT N` est ajoutée à la fin d'une requête pour calculer seulement les N premiers tuples du résultat.

Exemple pour afficher 10 tuples de la table Lineitem :

In [7]:
db.execute("SELECT * FROM lineitem LIMIT 10").df()

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,15519,785,1,17.0,24386.67,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,to beans x-ray carefull
1,1,6731,732,2,36.0,58958.28,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,according to the final foxes. qui
2,1,6370,371,3,8.0,10210.96,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,ourts cajole above the furiou
3,1,214,465,4,28.0,31197.88,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,s cajole busily above t
4,1,2403,160,5,24.0,31329.60,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,"the regular, regular pa"
5,1,1564,67,6,32.0,46897.92,0.07,0.02,N,O,1996-01-30,1996-02-07,1996-02-03,DELIVER IN PERSON,MAIL,rouches. special
6,2,10617,138,1,38.0,58049.18,0.00,0.05,N,O,1997-01-28,1997-01-14,1997-02-02,TAKE BACK RETURN,RAIL,re. enticingly regular instruct
7,3,430,181,1,45.0,59869.35,0.06,0.00,R,F,1994-02-02,1994-01-04,1994-02-23,NONE,AIR,s cajole above the pinto beans. iro
8,3,1904,658,2,49.0,88489.10,0.10,0.00,R,F,1993-11-09,1993-12-20,1993-11-24,TAKE BACK RETURN,RAIL,ecial pinto beans. sly
9,3,12845,370,3,27.0,47461.68,0.06,0.07,A,F,1994-01-16,1993-11-22,1994-01-23,DELIVER IN PERSON,SHIP,e carefully fina


La commande `SUMMARIZE` donne un aperçu statistique des données d'une table.
Elle retourne pour chaque attribut, les bornes min et max du domaine, le nombre approximatif de valeurs uniques ainsi que la distribution décrite par la moyenne, l'écart type et les quartiles 25, 50 (médiane) et 75.

In [8]:
db.execute("SUMMARIZE lineitem").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,l_orderkey,INTEGER,1,600000,156886,300087.32032628893,173380.15954532725,152438,300461,449237,600572,0.0%
1,l_partkey,INTEGER,1,20000,19915,10003.995747387491,5768.962517282622,5014,9999,15011,600572,0.0%
2,l_suppkey,INTEGER,1,1000,1010,500.55533391500103,288.5557626111972,251,501,750,600572,0.0%
3,l_linenumber,INTEGER,1,7,7,3.001215507882485,1.7321171926128305,1,3,4,600572,0.0%
4,l_quantity,"DECIMAL(15,2)",1.00,50.00,50,25.53366124294839,14.420188609142427,13,26,38,600572,0.0%
5,l_extendedprice,"DECIMAL(15,2)",901.00,95949.50,130790,35992.236201887536,22014.18174976397,17591,34469,51719,600572,0.0%
6,l_discount,"DECIMAL(15,2)",0.00,0.10,11,0.050073929520523766,0.0316031281910522,0,0,0,600572,0.0%
7,l_tax,"DECIMAL(15,2)",0.00,0.08,9,0.040041626982276896,0.025797304713675383,0,0,0,600572,0.0%
8,l_returnflag,VARCHAR,A,R,3,None,None,None,None,None,600572,0.0%
9,l_linestatus,VARCHAR,F,O,2,None,None,None,None,None,600572,0.0%


# Requêtes

On considère les requêtes suivantes qui sont des versions simplifiées d'une des requêtes du benchmark TPC-H :

In [10]:
query1 = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""


query2 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 50;
"""

query = query1

## Exécuter une requête

On définit la fonction `run_query` pour exécuter une requête et mesurer sa durée en millisecondes.

In [11]:
# par défaut on exécute la requête dans DuckDB (db)
def run_query(q, connection=db):
  start = time.time()
  connection.execute(q).fetchall()
  end = time.time()
  return(f"{(end - start)*1000:.0f}ms")


Exécuter la première requête dans DuckDB

In [12]:
run_query(query)

'10ms'

# Comparaison DuckDB versus SQLite

L'objectif est de comparer les performances de deux SGBD en mémoire :
DuckDB (très rapide) et SQLite (très populaire)

On veut montrer que DuckDB est plus rapide que SQLite pour traiter des requêtes "réalistes" de l'application TPC-H.


## Configuration de SQLite

Initialiser SQLite.

In [14]:
# sqlite = sqlite3.connect(':memory:', check_same_thread=False)
!rm -f tpch_sqlite.db
sqlite = sqlite3.connect('tpch_sqlite.db', check_same_thread=False)

On peut transférer les données depuis DuckDB vers SQLite par l'intermédaire d'un dataFrame pandas.
*   On exporte chaque table de DuckDB vers un dataframe en utilisant  la commande `.df()`
*   Puis on utilise la fonction `to_sql` d'un dataframe pandas pour générer les instructions SQL permettant d'importer les données dans SQLite.


In [15]:
table_list = db.execute("SHOW TABLES").fetchall()

# peut durer de 20s à  1 minute

for table in table_list:
  tname = table[0]
  table_data = db.table(tname).df()
  # table_data.to_sql(tname, sqlite)
  table_data.to_sql(tname, sqlite, if_exists='replace')


## Comparer les performances

On exécute la même requête dans les 2 SGBD: DuckDB et SQLite. Puis on compare les temps d'exécution.

In [16]:
print("On exécute la requête:", query)

duckdb_result = run_query(query, db)
sqlite_result = run_query(query, sqlite)

print("Les durées d'exécution sont: ")

pd.DataFrame.from_dict({
    'DuckDB': [duckdb_result],
    'SQLite': [sqlite_result]
})

On exécute la requête: 
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;

Les durées d'exécution sont: 


,DuckDB,SQLite
0,7ms,19ms


On constate que DuckDB est environ 20 fois plus rapide que SQLite pour cette requête.

Afin de montrer l'efficacité de l'optimiseur de requêtes,
on demande à DuckDB d'exécuter la requête sans l'optimiser.

La commande  `PRAGMA disable_optimizer` désactive l'optimiseur de requête.
Puis on re-exécute la requête pour mesurer sa durée.

In [17]:
db.execute("PRAGMA disable_optimizer")
duckdb_unoptimized_result = run_query(query)
db.execute("PRAGMA enable_optimizer")

pd.DataFrame.from_dict({
    'DuckDB': [duckdb_result],
    'DuckDB (sans optimisation de requête)': [duckdb_unoptimized_result],
    'SQLite': [sqlite_result]
})

,DuckDB,DuckDB (sans optimisation de requête),SQLite
0,7ms,43ms,19ms


On constate que la requête sans optimisation s'exécute beaucoup plus lentement (durée presque 5 fois plus longue!) que celle optimisée.

# Plan d'une requête
L'écart de durée entre la requête optimisée et celle non optimisée vient du fait qu'elles ont des plans d'exécution différents

On peut visualiser le **plan** d'exécution d'une requête en préfixant la requête par `EXPLAIN`.

La commande `EXPLAIN` ne fait qu'afficher le plan d'exécution ; elle n'exécute **pas** la requête.

Le plan affiché est appelé *plan physique* car il indique pour chaque opérateur du plan, le nom de l'algorithme utilisé:
*   HASH_GROUP_BY : regroupement par hachage puis agrégation
*   HASH_JOIN : jointure par hachage
*   FILTER : sélection en pipeline
*   SEQ_SCAN : lecture séquentielle d'une table

De plus le plan indique la cardinalité estimée **EC** pour certains opérateurs, en particulier pour les sélections et les jointures.

In [18]:
def explain_query(query):
  print(db.execute("EXPLAIN " + query).fetchall()[0][1])

print(query)
explain_query(query)


SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;

┌───────────────────────────┐                                                          
│           TOP_N           │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│           Top 50          │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴────

Remarque : vous pourriez invoquer `PRAGMA explain_output='all'` pour afficher, en plus du plan physique, le plan logique avant son optimisation et le plan logique optimisé, mais ce n'est pas demandé dans ce TP.

# Rapport d'exécution d'un plan

On peut afficher un rapport **après** l'exécution d'un plan pour connaitre la quantité de données traitées par chaque opération du plan. On appelle cela le *profil* de l'exécution.

La fonction  `run_and_profile_query` active le mode profil afin de sauvegarder les informations de profil dans un fichier.
La requête est ensuite exécutée puis les informations de profil produites pendant l'exécution sont affichées.

On peut lire dans le profil, pour chaque opérateur, la durée écoulée, le nombre de tuples consommés en entrée ainsi que le nombre de tuples produits pour l'opérateur suivant.

Plus spécifiquement, l'opérateur de lecture séquentielle `SEQ_SCAN` (sequential scan) indique le nombre de tuples lus dans une table.
L'opérateur `HASH_GROUP_BY` indique le nombre de groupes créés.

Remarque: pour les opérateurs qui matérialisent leur sortie, le nombre de tuples produits représente la cardinalité du résultat intermédiaire.

Les cardinalités intermédiaires sont importantes car elles peuvent expliquer pourquoi un opérateur a pris beaucoup de temps pour être traité.
Dans de nombreuses situations, il est possible de réduire la cardinalité des résultats intermédiaires et modifiant l'ordre des opérations et la façon dont elles sont traitées.


In [19]:
def run_and_profile_query(query):
  db.execute("PRAGMA enable_profiling")
  db.execute("PRAGMA profiling_output='out.log'")
  db.execute(query)
  db.execute("PRAGMA disable_profiling")
  with open('out.log', 'r') as f:
    output = f.read()
  print(output)

#run_and_profile_query(query)


# Exercice 1 : Optimiser un première requête

Pour mieux comprendre le fonctionnement de l'optimiseur de requêtes, on propose d'optimiser la requête "à la main" en SQL.
Pour cela, on désactive l'optimiseur de requêtes avec la commande `PRAGMA disable_optimizer`. Cela signifie que le SGBD va traduire la requête SQL dans un plan qui sera "le plus proche possible" de l'ordre des insctructions écrites dans la requête SQL.


## Requête à optimiser

In [20]:
query1_cross_product = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer,
    orders,
    lineitem
WHERE
    c_custkey=o_custkey
    AND l_orderkey=o_orderkey
    AND c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

query2_cross_product = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer,
    orders,
    lineitem
WHERE
    c_custkey=o_custkey
    AND l_orderkey=o_orderkey
    AND c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 50;
"""

Afficher le plan de la requête

In [21]:
db.execute("PRAGMA disable_optimizer")
explain_query(query1_cross_product)

# Note: ne pas executer cette requête car son plan est tres lent!

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│       HASH_GROUP_BY       │                                 

In [22]:
explain_query(query2_cross_product)

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│       sum((lineitem       │                                                          
│.l_extendedprice * (1 ...  │                                                          
│    .l_discount))) DESC    │                                                          
│   orders.o_orderdate ASC  │                                                          
└──

On voit que le plan ne contient aucune jointure mais des produits cartésiens suivis de sélections. Or les produits cartésiens sont coûteux à calculer!

On évite d'exécuter cette requête pendant le TP car cela  prendrait trop de temps...



On remplace les produits cartésiens par des jointures en ajoutant  des `JOIN` dans la clause `FROM` (comme dans query1).

Cela rend la requête plus rapide et on peut l'exécuter pour mesurer sa durée.

In [23]:
query_join = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

explain_query(query_join)
run_query(query_join)
run_and_profile_query(query_join)

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│       HASH_GROUP_BY       │                                 

## a) Traiter les sélections avant les jointures

Proposer d'autres écritures SQL équivalentes de la requête pour qu'elle soit traitée encore plus rapidement.

Utiliser des requêtes imbriquées pour pousser (ou non) les sélections et les projections.

Vérifier que la requête proposées est bien équivalente à la requête initiale
(avoir le même résultat est une condition nécessaire).



In [53]:
query_join_2 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 50;
"""


run_query(query_join_2)
# run_and_profile_query(query_join_2)

'52ms'

## b) Ordre des jointures

On peut voir que les tables sont jointes dans l'ordre Order,Customer, Lineitem

Re-ecrire la requête pour que les tables ne soient pas jointes dans des ordres différents. Est ce que cela change la durée de la requête ?

In [54]:
query_join_3 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    lineitem
    JOIN orders ON (l_orderkey=o_orderkey)
    JOIN customer ON (o_custkey=c_custkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 50;
"""

run_query(query_join_3)

'19ms'

## c) Index

Créer des index et montrer si cela a un effet sur la durée de la requête.

In [26]:
# Créer des index et montrer si cela a un effet sur la durée de la requête.
def create_index(table, column):
    db.execute(f"CREATE INDEX idx_{table}_{column} ON {table}({column})")

def del_all_index(list_index):
  for table, column in list_index:
    db.execute(f"DROP INDEX IF EXISTS idx_{table}_{column}")


## 创建索引
del_all_index([("customer", "c_custkey"),
               ("region", "r_regionkey"),
               ("nation", "n_regionkey"),
               ("supplier", "s_nationkey"),
               ("customer", "c_nationkey"),
               ("orders", "o_custkey"),
               ("lineitem", "l_orderkey"),
               ("lineitem", "l_shipdate")])
create_index("customer", "c_custkey")
create_index("region", "r_regionkey")
create_index("nation", "n_regionkey")
create_index("supplier", "s_nationkey")
create_index("customer", "c_nationkey")
create_index("orders", "o_custkey")
create_index("lineitem", "l_shipdate")

# Exercice 2 : Optimiser les requêtes TPCH

Optimiser chaque requête de TPC-H.
Vous pouvez les obtenir avec `SELECT * FROM tpch_queries()`


In [27]:
queries = db.execute("SELECT * FROM tpch_queries()").fetchall()

# for id, sql in queries:
for id, sql in queries[:5]:
  print('### requete ', id, ':')
  print(sql)

### requete  1 :
SELECT
    l_returnflag,
    l_linestatus,
    sum(l_quantity) AS sum_qty,
    sum(l_extendedprice) AS sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    count(*) AS count_order
FROM
    lineitem
WHERE
    l_shipdate <= CAST('1998-09-02' AS date)
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;

### requete  2 :
SELECT
    s_acctbal,
    s_name,
    n_name,
    p_partkey,
    p_mfgr,
    s_address,
    s_phone,
    s_comment
FROM
    part,
    supplier,
    partsupp,
    nation,
    region
WHERE
    p_partkey = ps_partkey
    AND s_suppkey = ps_suppkey
    AND p_size = 15
    AND p_type LIKE '%BRASS'
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'EUROPE'
    AND ps_supplycost = (
      

In [59]:
### requete 1 :
req_TPCH_1="""
SELECT
    l_returnflag,
    l_linestatus,
    sum(l_quantity) AS sum_qty,
    sum(l_extendedprice) AS sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    count(*) AS count_order
FROM
    lineitem
WHERE
    l_shipdate <= CAST('1998-09-02' AS date)
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
"""

req_TPCH_1_opt="""
SELECT
    l_returnflag,
    l_linestatus,
    sum(l_quantity) AS sum_qty,
    sum(l_extendedprice) AS sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    count(*) AS count_order
FROM
    (SELECT l_returnflag, l_linestatus, l_quantity, l_extendedprice, l_discount, l_tax, l_quantity FROM lineitem WHERE l_shipdate <= CAST('1998-09-02' AS date))
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
"""

print(run_query(req_TPCH_1_opt))

22ms
10ms


In [104]:
### requete  2 :
req_TPCH_2="""
SELECT
    s_acctbal,
    s_name,
    n_name,
    p_partkey,
    p_mfgr,
    s_address,
    s_phone,
    s_comment
FROM
    part,
    supplier,
    partsupp,
    nation,
    region
WHERE
    p_partkey = ps_partkey
    AND s_suppkey = ps_suppkey
    AND p_size = 15
    AND p_type LIKE '%BRASS'
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'EUROPE'
    AND ps_supplycost = (
        SELECT
            min(ps_supplycost)
        FROM
            partsupp,
            supplier,
            nation,
            region
        WHERE
            p_partkey = ps_partkey
            AND s_suppkey = ps_suppkey
            AND s_nationkey = n_nationkey
            AND n_regionkey = r_regionkey
            AND r_name = 'EUROPE')
ORDER BY
    s_acctbal DESC,
    n_name,
    s_name,
    p_partkey
LIMIT 100;
"""

req_TPCH_2_opt="""
SELECT
    s_acctbal,
    s_name,
    n_name,
    p_partkey,
    p_mfgr,
    s_address,
    s_phone,
    s_comment
FROM
    partsupp
    JOIN (SELECT * FROM part WHERE p_size = 15 AND p_type LIKE '%BRASS') ON (ps_partkey=p_partkey)
    JOIN supplier ON (ps_suppkey=s_suppkey)
    JOIN nation ON (s_nationkey=n_nationkey)
WHERE
    ps_supplycost = (
        SELECT
            min(ps_supplycost)
        FROM
            (SELECT * FROM region WHERE r_name = 'EUROPE')
            JOIN nation ON (r_regionkey=n_regionkey)
            JOIN supplier ON (n_nationkey=s_nationkey)
            JOIN partsupp ON (s_suppkey=ps_suppkey)
    )
ORDER BY
    s_acctbal DESC,
    n_name,
    s_name,
    p_partkey
LIMIT 100;
"""

print(run_query(req_TPCH_2_opt))

7ms
11ms


In [105]:
### requete  3 :
req_TPCH_3="""
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer,
    orders,
    lineitem
WHERE
    c_mktsegment = 'BUILDING'
    AND c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""
req_TPCH_3_opt="""
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    (SELECT * from lineitem WHERE l_shipdate > CAST('1995-03-15' AS date))
    JOIN (SELECT * FROM orders WHERE o_orderdate < CAST('1995-03-15' AS date)) ON (o_orderkey=l_orderkey)
    JOIN (SELECT * FROM customer WHERE c_mktsegment = 'BUILDING') ON (o_custkey=c_custkey)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""

print(run_query(req_TPCH_3_opt))

34ms


In [106]:
### requete  4 :
req_TPCH_4="""
SELECT
    o_orderpriority,
    count(*) AS order_count
FROM
    orders
WHERE
    o_orderdate >= CAST('1993-07-01' AS date)
    AND o_orderdate < CAST('1993-10-01' AS date)
    AND EXISTS (
        SELECT
            *
        FROM
            lineitem
        WHERE
            l_orderkey = o_orderkey
            AND l_commitdate < l_receiptdate)
GROUP BY
    o_orderpriority
ORDER BY
    o_orderpriority;
"""

req_TPCH_4_opt="""
SELECT
    o_orderpriority,
    count(*) AS order_count
FROM
    (SELECT * FROM lineitem WHERE l_commitdate < l_receiptdate)
    JOIN (SELECT * FROM orders WHERE o_orderdate >= CAST('1993-07-01' AS date) AND o_orderdate < CAST('1993-10-01' AS date)) ON (o_orderkey = l_orderkey)
GROUP BY
    o_orderpriority
ORDER BY
    o_orderpriority;
"""

print(run_query(req_TPCH_4_opt))

17ms


In [123]:
### requete  5 :
# Optimiser chaque requête de TPC-H.
req_TPCH_5="""
SELECT
    n_name,
    sum(l_extendedprice * (1 - l_discount)) AS revenue
FROM
    customer,
    orders,
    lineitem,
    supplier,
    nation,
    region
WHERE
    c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND l_suppkey = s_suppkey
    AND c_nationkey = s_nationkey
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND o_orderdate >= CAST('1994-01-01' AS date)
    AND o_orderdate < CAST('1995-01-01' AS date)
GROUP BY
    n_name
ORDER BY
    revenue DESC;"""

req_TPCH_5_opt="""
SELECT
    n_name,
    sum(l_extendedprice * (1 - l_discount)) AS revenue
FROM
    (SELECT * FROM region WHERE r_name='ASIA')
    JOIN nation ON (r_regionkey=n_regionkey)
    JOIN customer ON (n_nationkey=c_nationkey)
    JOIN (SELECT * FROM orders WHERE o_orderdate >= CAST('1994-01-01' AS date) AND o_orderdate < CAST('1995-01-01' AS date)) ON (c_custkey=o_custkey)
    JOIN lineitem ON (o_orderkey=l_orderkey)
    JOIN supplier ON (n_nationkey=s_nationkey)
GROUP BY
    n_name
ORDER BY
    revenue DESC;"""

run_query(req_TPCH_5_opt)

'54ms'

# Exercice 3 : Cardinalités

On considère le plan (optimisé) de la requête `query1`.

Expliquer comment est calculée chaque valeur de cardinalité estimée **EC**.

 à compléter

# Exercice 4 : Optimiser une requête avec 6 tables

Soit la requête suivante qui accède à 6 tables.

In [33]:
query_6_tables = """
SELECT
    nation,
    o_year,
    sum(amount) AS sum_profit
FROM (
    SELECT
        n_name AS nation,
        extract(year FROM o_orderdate) AS o_year,
        l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity AS amount
    FROM
        part,
        supplier,
        lineitem,
        partsupp,
        orders,
        nation
    WHERE
        s_suppkey = l_suppkey
        AND ps_suppkey = l_suppkey
        AND ps_partkey = l_partkey
        AND p_partkey = l_partkey
        AND o_orderkey = l_orderkey
        AND s_nationkey = n_nationkey
        AND p_name LIKE '%green%') AS profit
GROUP BY
    nation,
    o_year
ORDER BY
    nation,
    o_year DESC;
"""

## a) Enumérer les ordres de jointure
Combien d'ordre de jointures sont possibles ?

à compléter

6 * 5 * 4 * 3 * 2 * 1 = 720

## b) Requête optimale
 Re-*écrire* cette requête pour qu'elle s'exécute le plus rapidement possible.

In [42]:
query_6_tables_opt2 =  """
SELECT
    n_name,
    o_year,
    sum(l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity) AS sum_profit
FROM 
    (SELECT extract(year FROM o_orderdate) AS o_year, o_orderkey FROM orders)
    JOIN lineitem ON (o_orderkey=l_orderkey)
    JOIN partsupp ON (l_suppkey=ps_suppkey AND l_partkey=ps_partkey)
    JOIN (SELECT * FROM part WHERE p_name LIKE '%green%') ON (l_partkey=p_partkey)
    JOIN supplier ON (l_suppkey=s_suppkey)
    JOIN nation ON (s_nationkey=n_nationkey)
GROUP BY
    n_name,
    o_year
ORDER BY
    n_name,
    o_year DESC;
"""
print(run_query(query_6_tables_opt2))


73ms
